# Request Struct and Response Struct for fi

In [ ]:
// Result message
const (
	OK       = "OK"
	ErrNoKey = "ErrNoKey"
)

type Err string

// struct of put
type PutReq struct {
	Key   string
	Value string
}

type PutResp struct {
	Err Err
}

// struct of get
type GetReq struct {
	Key string
}

type GetResp struct {
	Err   Err
	Value string
}

# Client: fi, rpc_client

In [ ]:
func get(key string) string {
	// Dial
	rpc_client, err := rpc.Dial("tcp", ":1234")
	if err != nil {
		log.Fatal("dialing:", err)
	}

	// Get req and resp
	req := GetReq{key}
	resp := GetResp{}

	// Call (waiting for serveConn)
	err2 := rpc_client.Call("KV.Get", &req, &resp)
	if err2 != nil {
		log.Fatal("error:", err)
	}

	// Close
	rpc_client.Close()
	return resp.Value
}

func put(key string, val string) {
	rpc_client, err := rpc.Dial("tcp", ":1234")
	if err != nil {
		log.Fatal("dialing:", err)
	}
	req := PutReq{key, val}
	resp := PutResp{}
	err2 := rpc_client.Call("KV.Put", &req, &resp)
	if err2 != nil {
		log.Fatal("error:", err)
	}
	rpc_client.Close()
}

# Server: Fi, rpc_server, kv, listener

In [ ]:
type KV struct {
	mu   sync.Mutex
	data map[string]string
}

func server() {
	// Create a new KV -- kv
	kv := KV{}
	kv.data = map[string]string{}

	// Create a new rpc_server and register methods for kv
	rpcs := rpc.NewServer()
	rpcs.Register(kv)

	// Create a new listener
	l, e := net.Listen("tcp", ":1234")
	if e != nil {
		log.Fatal("listen error:", e)
	}

	// This go func() is to make clients, which run after the server() in main(), run
	// we can ignore it when running client and server in different process
	go func() {
		for {
			// Accept (blocked and waiting for Dial)
			conn, err := l.Accept()
			if err == nil {
				// create a new thread to serve the connection (blocked and waiting for rpc_client.Call)
				go rpcs.ServeConn(conn)
			} else {
				break
			}
		}
		l.Close()
	}()
}

func (kv KV) Get(req *GetReq, resp *GetResp) error {
	kv.mu.Lock()
	defer kv.mu.Unlock()

	// req -> args
	// f(args) -> res
	val, ok := kv.data[req.Key] // found or not
	// res -> resp
	if ok {
		resp.Err = OK
		resp.Value = val
	} else {
		resp.Err = ErrNoKey
		resp.Value = ""
	}
	return nil
}

func (kv KV) Put(req *PutReq, resp *PutResp) error {
	kv.mu.Lock()
	defer kv.mu.Unlock()

	kv.data[req.Key] = req.Value
	resp.Err = OK
	return nil
}